In [1]:
# check that 'contact' and 'resolution_code' do not contain "SPAM SPAM" or "Spam", respectively 
import pandas as pd 
import numpy as np
import re

In [80]:
# INDIVIDUAL TESTING

# "SNOW July Data.csv" is the July data pull from SNOW

july_data = pd.read_csv("SNOW July Data.csv", encoding = "ISO-8859-1")
july_data = july_data.loc[(july_data['contact'] != 'SPAM SPAM') & (july_data['resolution_code'] != 'Spam')].reset_index(drop=True)

# currently selects descriptions manually. This will later be transitioned to a for-loop or some function
# look at july_data to see which index value you want to see tested
july_samp = july_data.iloc[234]['description']

# DESCRIPTION CLEANING
regex_sol = re.sub(r'https?://\S+', '', july_samp) # removes URL links
regex_sol = re.sub(r"\S*@\S*\s?", "", regex_sol) # removes email accounts
regex_sol = regex_sol.replace("\n", "").replace("< >","").replace("\r", "") # removes newline and <> and \r
regex_sol = re.sub(r"\d+", "", regex_sol) # removes integers 
regex_sol = re.sub(r"([^\s\w]|_)+", " ", regex_sol) # removes non-alphanumeric characters, but maintains whitespace
regex_sol = regex_sol.encode("ASCII", "replace").decode("utf-8").replace("?", " ") # removes all non-ASCII characters
regex_sol = regex_sol.lower()
# regex_sol

# SUBJECT LINE CLEANING
july_samp_2 = july_data.iloc[7]['case']
regex_sol_2 = re.sub(r"[?](CS)[0-9]+", "", july_samp_2)
regex_sol_2 = re.sub(r"\d+", "", regex_sol_2)
regex_sol_2 = re.sub(r"([^\s\w]|_)+", " ", regex_sol_2)
regex_sol_2 = regex_sol_2.lower()
# regex_sol_2


'This morning I noticed a change in the Goods - Multiple Shipment form that seems unusual.  For line type the goods form states "Services" and not "Goods".  Also it does not allow me to select a quantity nor a Unit of Measure (UOM).  Attaching a screen shot for reference.'

In [91]:
# FUNCTIONS APPLY CURRENT CLEANING STEPS INTO ENTIRE DATASET

def subject_clean(subject_line):
    regex_sol = re.sub(r'https?://\S+', '', subject_line) # removes URL links
    regex_sol = re.sub(r"\S*@\S*\s?", "", regex_sol) # removes email accounts
    regex_sol = regex_sol.replace("\n", "").replace("< >","").replace("\r", "") # removes newline and <> and \r
    regex_sol = re.sub(r"\d+", "", regex_sol) # removes integers 
    regex_sol = re.sub(r"([^\s\w]|_)+", " ", regex_sol) # removes non-alphanumeric characters, but maintains whitespace
    regex_sol = regex_sol.encode("ASCII", "replace").decode("utf-8").replace("?", " ") # removes all non-ASCII characters
    regex_sol = regex_sol.lower() # lower case string
    return regex_sol

def case_line_clean(case_line):
    regex_sol_2 = re.sub(r"[?](CS)[0-9]+", "", case_line) # removes ending "?CS###"
    regex_sol_2 = re.sub(r"\d+", "", regex_sol_2) # removes integers
    regex_sol_2 = re.sub(r"([^\s\w]|_)+", " ", regex_sol_2) # removes non-characters, but maintains whitespace
    regex_sol_2 = regex_sol_2.lower()
    return regex_sol_2

# call the dataset that pertains to you
july_dataset = pd.read_csv("SNOW July Data.csv", encoding = "ISO-8859-1")
june_dataset = pd.read_csv("SNOW June Data.csv", encoding = "ISO-8859-1")

#SPAM case removal
july_dataset = july_dataset.loc[(july_dataset['contact'] != 'SPAM SPAM') & (july_dataset['resolution_code'] != 'Spam')].reset_index(drop=True)
june_dataset = june_dataset.loc[(june_dataset['contact'] != 'SPAM SPAM') & (june_dataset['resolution_code'] != 'Spam')].reset_index(drop=True)

july_dataset['description'] = july_dataset['description'].apply(subject_clean)
june_dataset['description'] = june_dataset['description'].apply(subject_clean)

july_dataset['case'] = july_dataset['case'].apply(case_line_clean)
june_dataset['case'] = june_dataset['case'].apply(case_line_clean)

In [93]:
july_dataset

,case,contact,description,short_description,business_service,service_offering,resolution_code
0,add a requisition line for shopping,Janelle Fine,hello i need to make a purchase that would ha...,"""Add a Requisition Line"" for shopping",Oracle Procurement,How to Shop,Inconclusive - No Customer Response
1,enter requisition line missing from more ta...,Alex Clark,enter requisition line for buying a specific...,"""Enter Requisition Line"" missing from 'more ta...",Oracle Procurement,How to Shop,Inconclusive - No Customer Response
2,d room number e projects charge acct s ...,Irina Kufareva,this is report following cs d when i edit...,"(d) room number, (e) projects & charge acct's,...",Oracle Procurement,How to Shop,NaN
3,urgent change order errors,Kathleen Au,hello lab members and i have been referring to...,(Urgent!) Change Order Errors,Oracle Procurement,Other Buying Inquiry,NaN
4,urgent sequencing services in oracle,Kathleen Au,hello we ve noticed that the sequencing servic...,(Urgent!) Sequencing Services in Oracle,Oracle Procurement,Punchouts,NaN
...,...,...,...,...,...,...,...
1035,your requisition preferences are invalid fix ...,Nina Hill,your requisition preferences are invalid fix ...,Your requisition preferences are invalid. Fix ...,Oracle Procurement,Approvals on Requisitions,NaN
1036,national instruments tmd document purchase...,ni ni,received from ni company name universit...,[National Instruments] (TMD-2614068) Document ...,Oracle Procurement,Other Buying Inquiry,Inconclusive - No Customer Response
1037,national instruments tmd document purchase...,NI Customer Service Team,received from ni company name universit...,[National Instruments] (TMD-2614068) Document ...,Oracle Procurement,Other Buying Inquiry,Inconclusive - No Customer Response
1038,national instruments tmd document purchase...,NI Customer Service Team,received from ni company name universit...,[National Instruments] (TMD-2614068) Document ...,Oracle Procurement,Other Buying Inquiry,Inconclusive - No Customer Response
